In [11]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.offline as pyo

In [2]:
#region_name = "Ridgecrest"
region_name = "testing"
stations = pd.read_csv(f"./{region_name}/stations.csv")

xlim = (-117.8, -117.3)
ylim = (35.5, 36.0)
zlim = (0, 15)
zlim_special = (0, 20)


x0 = stations["longitude"].median()
y0 = stations["latitude"].median()
xmin = stations["longitude"].min()
xmax = stations["longitude"].max()
ymin = stations["latitude"].min()
ymax = stations["latitude"].max()

xlim = (2 * xmin - x0, 2 * xmax - x0)
ylim = (2 * ymin - y0, 2 * ymax - y0)

zlim_special = (0, 20)
zlim= (0, 250)
size = 1.0
alpha = 0.5
max_sigma = 0.6

# catalog_hypoinverse = pd.read_csv("catOut.sum", sep="\s+")
catalog_hypoDD = pd.read_csv(f"./{region_name}/hypoDD_catalog.txt", sep="\s+", names=["ID", "LAT", "LON", "DEPTH", "X", "Y", "Z", "EX", "EY", "EZ", "YR", "MO", "DY", "HR", "MI", "SC", "MAG", "NCCP", "NCCS", "NCTP",
"NCTS", "RCC", "RCT", "CID"])

catalog_hypoDD["time"] = catalog_hypoDD.apply(lambda x: f'{x["YR"]:04.0f}-{x["MO"]:02.0f}-{x["DY"]:02.0f}T{x["HR"]:02.0f}:{x["MI"]:02.0f}:{min(x["SC"], 59.999):05.3f}', axis=1)
catalog_hypoDD["time"] = catalog_hypoDD["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f"))

# catalog_gamma = pd.read_csv(f"./{region_name}/gamma_catalog.csv", sep="\t")
# picks_gamma = pd.read_csv(f"./{region_name}/gamma_picks.csv", sep="\t")

In [5]:
c_gamma = (np.array(catalog_hypoDD["time"]) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
c_hypodd = (np.array(catalog_hypoDD["time"]) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
t_gamma = catalog_hypoDD["time"]
t_hypodd = catalog_hypoDD["time"]

C:\Users\acer\AppData\Local\Temp\ipykernel_3996\1870006410.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  c_gamma = (np.array(catalog_hypoDD["time"]) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
C:\Users\acer\AppData\Local\Temp\ipykernel_3996\1870006410.py:2: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  c_hypodd = (np.array(catalog_hypoDD["time"]) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')


In [13]:
x = catalog_hypoDD["LON"]
y = catalog_hypoDD["LAT"]
z = catalog_hypoDD["DEPTH"]

c_depth = z.copy()
c_depth[c_depth<zlim[0]] = zlim[0]
c_depth[c_depth>zlim[1]] = zlim[1]

c_time = c_hypodd

In [14]:
fig = go.Figure(
    data=[go.Scatter3d(x=x, y=y, z=z,
        mode='markers',
        marker=dict(size=size,
            color=c_time,                 
            colorscale='Viridis',  
            opacity=0.8))],
            )

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=xlim,),
        yaxis = dict(nticks=4, range=ylim,),
        zaxis = dict(nticks=4, range=zlim,),
#         aspectratio = dict(x=(xlim[1]-xlim[0])/2, y=(ylim[1]-ylim[0])/2, z=1),
        aspectratio = dict(x=1, y=1, z=0.5),
    ),
    margin=dict(r=0, l=0, b=0, t=0),
)
fig.write_html(f"{region_name}/catalog_color_by_time.html")
fig.show()

In [15]:
fig = go.Figure(
    data=[go.Scatter3d(x=x, y=y, z=z,
        mode='markers',
        marker=dict(size=size,
            color=c_depth,                 
            colorscale='Viridis',  
            opacity=0.8))],
            )

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=xlim,),
        yaxis = dict(nticks=4, range=ylim,),
        zaxis = dict(nticks=4, range=zlim,),
#         aspectratio = dict(x=(xlim[1]-xlim[0])/2, y=(ylim[1]-ylim[0])/2, z=1),
        aspectratio = dict(x=1, y=1, z=0.5),
    ),
    margin=dict(r=0, l=0, b=0, t=0),
)
fig.write_html(f"{region_name}/catalog_color_by_depth.html")
fig.show()